In [ ]:
# Instalando as bibliotecas necessárias
!pip install biopython pandas requests

# Importando os módulos necessários
from tkinter import filedialog,messagebox
from Bio import Entrez,Phylo
import tkinter as tk
import pandas as pd
import requests
import logging
import time
import io


In [ ]:
def configure_entrez():
    """
    Configura as credenciais para o NCBI Entrez.

    Solicita ao usuário que forneça as configurações do NCBI Entrez e as define.
    O usuário pode fornecer a chave da API e o e-mail associado. Caso não forneça,
    serão utilizados valores preestabelecidos.

    Parâmetros:
    default_api_key (str): Chave da API padrão para o NCBI Entrez.
    default_email (str): E-mail padrão para o NCBI Entrez.
    """
    api_key = input(f"Por favor, insira a sua chave da API do NCBI Entrez: ")
    email = input(f"Por favor, insira o seu e-mail para uso com o NCBI Entrez: ")

    Entrez.api_key = api_key
    Entrez.email = email

    logging.info(f"Configurações do NCBI Entrez definidas. Chave da API: {api_key}, E-mail: {email}")

    print("Configurações do NCBI Entrez definidas com sucesso!")

def search_NCBI(search_for, max_retries=3):
    """
    Realiza uma busca no NCBI usando o termo especificado.

    Parâmetros:
    search_for (str): Termo a ser pesquisado.
    max_retries (int): Número máximo de tentativas de consulta.

    Retorna:
    dict: Resultado da busca no formato de dicionário.
    """
    for tentativa in range(max_retries + 1):
        try:
            handle = Entrez.esearch(db='taxonomy', term=search_for)
            result = Entrez.read(handle)
            handle.close()
            return result
        except Exception as e:
            logging.error(f"Erro ao realizar a busca no NCBI para '{search_for}' na tentativa {tentativa+1}: {e}")
            if tentativa == max_retries:
                return None

def efetch_NCBI(efetch_for, max_retries=3):
    """
    Realiza uma busca no NCBI usando o termo especificado.

    Parâmetros:
    efetch_for (str): Termo a ser pesquisado.
    max_retries (int): Número máximo de tentativas de consulta.

    Retorna:
    dict: Resultado da busca no formato de dicionário.
    """
    for tentativa in range(max_retries + 1):
        try:
            stream = Entrez.efetch(db='taxonomy', id=efetch_for)
            result = Entrez.read(stream)
            stream.close()
            return result
        except Exception as e:
            logging.error(f"Erro ao realizar a busca no NCBI para '{efetch_for}' na tentativa {tentativa+1}: {e}")
            if tentativa == max_retries:
                return None

def search_tax_id(scientific_name, max_retries=3):
    """
    Realiza uma busca pelo ID de taxonomia de um nome científico.

    Parâmetros:
    scientific_name (str): Nome científico da espécie.
    max_retries (int): Número máximo de tentativas de consulta.

    Retorna:
    str: O ID de taxonomia da espécie.
    """
    result = search_NCBI(scientific_name, max_retries)
    if result:
        txid = ','.join(result['IdList'])
        if txid:
            logging.info(f"ID de taxonomia obtido com sucesso para '{scientific_name}'.")
            return txid
        else:
            logging.warning(f"Nome científico '{scientific_name}' não encontrado na pesquisa.")
            return None
    else:
        logging.error(f"Erro ao buscar ID de taxonomia para '{scientific_name}'.")
        return None

def fetch_tax_info(txid, max_retries=3):
    """
    Obtém informações de taxonomia para um determinado ID de taxonomia.

    Parâmetros:
    txid (str): O ID de taxonomia da espécie.
    max_retries (int): Número máximo de tentativas de consulta.

    Retorna:
    tuple: Uma tupla contendo o nome científico, a classificação taxonômica
    e o ID do taxonômico superior.
    """
    record = efetch_NCBI(txid, max_retries)
    if record:
        name = record[0]['ScientificName']
        rank = record[0]['Rank']
        taxid_sup_rank = record[0]['LineageEx'][-1]['TaxId']
        logging.info(f"Informações de taxonomia obtidas com sucesso para '{name}'.")
        return name, rank, taxid_sup_rank
    else:
        logging.error(f"Erro ao obter informações de taxonomia para o ID '{txid}'.")
        return None, None, None

def get_taxonomy_id(scientific_name, max_retries=3):
    """
    Obtém informações de taxonomia para um nome científico.

    Parâmetros:
    scientific_name (str): Nome científico da espécie.
    max_retries (int): Número máximo de tentativas de consulta.

    Retorna:
    tuple: Uma tupla contendo o nome científico, o ID de taxonomia, a classificação
    taxonômica e o ID do taxonômico superior, além de uma string vazia ou com os nomes de espécies que 
    não foram encontradas.
    """
    errors = ''
    txid = search_tax_id(scientific_name, max_retries)
    if txid:
        name, rank, taxid_sup_rank = fetch_tax_info(txid, max_retries)
        time.sleep(0.1)
        return name, txid, rank, taxid_sup_rank, errors
    else:
        errors = scientific_name
        time.sleep(0.1)
        return None, None, None, None, errors

def salvar_arquivo(extensao='.csv'):
    """
    Abre uma janela de seleção de arquivo para o usuário escolher o local e o nome do arquivo CSV.

    Retorna:
    str: O caminho completo do arquivo a ser salvo.
    """
    print("Abrindo caixa de seleção para salvar o arquivo. Por favor, verifique se a janela não está oculta.")
    # Criar uma instância da janela principal do Tkinter e ocultá-la
    root = tk.Tk()
    root.withdraw()

    # Exibir uma mensagem na caixa de seleção de arquivos
    caminho_arquivo = filedialog.asksaveasfilename(
        defaultextension=extensao,
        # filetypes=[("Arquivos CSV", "*.csv")],
        title="Salvar Arquivo"
    )
    return caminho_arquivo

def selecionar_arquivo():
    """
    Abre uma janela de seleção de arquivo para o usuário escolher um arquivo CSV.

    Retorna:
    str: O caminho completo do arquivo selecionado.
    """
    print("Abrindo caixa de seleção de arquivo. Por favor, verifique se a janela não está oculta.")

    # Criar uma instância da janela principal do Tkinter e ocultá-la
    root = tk.Tk()
    root.withdraw()

    # Exibir uma mensagem na caixa de seleção de arquivos
    arquivo_csv = filedialog.askopenfilename(
        title="Selecione um arquivo",
        filetypes=[("Arquivo", "*.*")],
        multiple=False
    )

    # Retornar o caminho do arquivo selecionado
    return arquivo_csv

def ler_arquivo_csv():
    """
    Lê o arquivo CSV selecionado e verifica se contém a coluna 'scientificName'.

    Retorna:
    DataFrame: O DataFrame contendo os dados do arquivo CSV se a coluna 'scientificName' estiver presente, caso contrário, None.
    """
    arquivo_csv = selecionar_arquivo()
    try:
        logging.info(f"Lendo o arquivo CSV: {arquivo_csv}")
        df = pd.read_csv(arquivo_csv)
        if 'scientificName' not in df.columns:
            logging.error(f"O arquivo selecionado {arquivo_csv} não contém a coluna 'scientificName'.")
            messagebox.showerror("Erro", "O arquivo selecionado não contém a coluna 'scientificName'. Por favor, selecione outro arquivo.")
            # Chama recursivamente a função para selecionar um novo arquivo
            return ler_arquivo_csv()
        else:
            logging.info(f"Coluna 'scientificName' encontrada no arquivo {arquivo_csv}.")
            return df
    except Exception as e:
        logging.error(f"Erro ao ler o arquivo CSV: {e}")
        messagebox.showerror("Erro", f"Erro ao ler o arquivo CSV: {e}")

def get_children(tax_hierar, nome_cientifico_col='scientificName', filhos_col='index_filho', root=any, arvore={}):
    """
    Função recursiva para obter os filhos de um nó na hierarquia taxonômica.

    Parâmetros:
    tax_hierar (DataFrame): O DataFrame contendo a hierarquia taxonômica.
    nome_cientifico_col (str): O nome da coluna que contém os nomes científicos dos táxons.
    filhos_col (str): O nome da coluna que contém os índices dos filhos de cada táxon.
    root (any): O nó raiz para começar a construção da árvore. Padrão: qualquer valor.
    arvore (dict): O dicionário que representa a árvore. Padrão: {}.

    Retorna:
    None
    """
    indices_filho = tax_hierar.loc[tax_hierar[nome_cientifico_col] == root, filhos_col].values
    if len(indices_filho) > 0:
        indices_filho = indices_filho[0]
        if pd.notna(indices_filho):
            indices_filho = indices_filho.split(',')
            indices_filho = [int(i) for i in indices_filho]
            filhos_raiz = tax_hierar.loc[indices_filho, nome_cientifico_col]
            arvore[root] = {filho for filho in filhos_raiz}

            # Adicionar os filhos à raiz correspondente no dicionário arvore
            for filho in filhos_raiz:
                get_children(tax_hierar, root=filho)

def construir_arvore_taxonomica(tax_hierar, nome_cientifico_col='scientificName', filhos_col='index_filho', root=any):
    """
    Constrói uma árvore taxonômica a partir dos dados de hierarquia taxonômica fornecidos.

    Parâmetros:
    tax_hierar (DataFrame): O DataFrame contendo a hierarquia taxonômica.
    nome_cientifico_col (str): O nome da coluna que contém os nomes científicos dos táxons.
    filhos_col (str): O nome da coluna que contém os índices dos filhos de cada táxon.
    root (any): O nó raiz ou uma lista de nós raiz para começar a construção da árvore. Padrão: qualquer valor.

    Retorna:
    dict: Um dicionário representando a árvore taxonômica.
    """
    # Inicializar a árvore
    arvore = {}
    
    # Inicializar a árvore com a raiz 'Eukaryota'
    arvore['Eukaryota'] = {'Metazoa', 'Viridiplantae'}

    # Chamar a função get_children para construir a árvore a partir das raízes fornecidas
    for raiz in root:
        get_children(tax_hierar, root=raiz, arvore=arvore)

    return arvore

def arvore_para_newick(arvore, no_raiz):
    """
    Converte uma árvore representada como um dicionário em formato Newick.

    Parâmetros:
    arvore (dict): O dicionário representando a árvore.
    no_raiz (str): O nó raiz da árvore.

    Retorna:
    str: A representação Newick da árvore.
    """
    if no_raiz not in arvore:
        return no_raiz
    else:
        filhos = arvore[no_raiz]
        newick = "("
        # Converte recursivamente os filhos em formato Newick
        for filho in filhos:
            newick += arvore_para_newick(arvore, filho) + ","
        newick = newick[:-1] + ")"  # Remove a última vírgula e adiciona parênteses de fechamento
        return newick + no_raiz

# Esta URL e a string 'rank_dwc' precisam ser explicadas
url_api = 'http://bioinfo.icb.ufmg.br/cgi-bin/taxallnomy/taxallnomy_multi.pl'
rank_dwc = 'Kingdom, Phylum, Class, Order, Superfamily, Family, Subfamily, Genus, Subgenus, Species'


In [ ]:
# Trocar scientificName na coluna
# Configurando o sistema de registro para gravar mensagens em um arquivo de log
# caminho_logfile = salvar_arquivo('.log')
logging.basicConfig(filename='logfile.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
configure_entrez()

In [ ]:
# Lê o arquivo CSV e armazena os dados em um DataFrame
df = ler_arquivo_csv()

if not df.empty:
    # Copiando os dados de df_scientificname para tax_hierar
    df_scientificname = df.drop_duplicates(subset='scientificName')['scientificName']
    df_scientificname = pd.DataFrame(df_scientificname)
    tax_hierar = df_scientificname.copy()
else:
    # Criando um DataFrame vazio
    tax_hierar = pd.DataFrame(columns=['taxonID', 'taxon', 'ID_pai', 'parent_taxon_index', 
                                       'ID_filho', 'index_filho', 'scientificName_correct', 'scientificName'])


In [ ]:

# Adicionando colunas vazias para armazenar informações de taxonomia
tax_hierar['taxonID'] = None
tax_hierar['taxon'] = None
tax_hierar['ID_pai'] = None
tax_hierar['parent_taxon_index'] = None
tax_hierar['ID_filho'] = None
tax_hierar['index_filho'] = None
tax_hierar['scientificName_correct'] = None

# Dividindo a coluna 'scientificName' em listas separadas por ' | '
tax_hierar['scientificName'] = tax_hierar['scientificName'].str.split(' \| ')

# Expandindo as listas para linhas individuais no DataFrame
tax_hierar = tax_hierar.explode('scientificName')

# Removendo entradas duplicadas de 'scientificName'
tax_hierar.drop_duplicates(subset='scientificName', inplace=True, ignore_index=True)

# Adicionando mensagem de log para o término da operação
logging.info("Preparação do DataFrame 'tax_hierar' concluída com sucesso.")


In [ ]:

# Lista para armazenar nomes de espécies que não puderam ser encontradas na pesquisa de taxonomia
lista_erro = []

# Iterando sobre os valores da coluna 'scientificName' do DataFrame 'tax_hierar'
for i, valor in enumerate(tax_hierar['scientificName']):
    # Chamando a função 'get_taxonomy_id' para obter informações de taxonomia para o valor atual
    tax_hierar.loc[i, 'scientificName_correct'], tax_hierar.loc[i, 'taxonID'], tax_hierar.loc[i, 'taxon'], tax_hierar.loc[i, 'ID_pai'], erro = get_taxonomy_id(valor)

    # Verificando se ocorreu um erro durante a busca de taxonomia
    if erro:
        lista_erro.append(erro)
        logging.warning(f"Erro ao obter informações de taxonomia para '{valor}': Nome não encontrado na pesquisa.")

tax_hierar_taxonid_none = tax_hierar[tax_hierar['taxonID'].isna()]

# Removendo linhas onde 'taxonID' está vazio, indicando que a busca de taxonomia não teve êxito
tax_hierar.dropna(subset='taxonID',inplace=True)
logging.info("Informações de taxonomia obtidas com sucesso para todas as espécies.")

# Resetando o índice do DataFrame após a remoção de linhas
tax_hierar.reset_index(drop=True, inplace=True)

# Adicionando mensagem de log para o término da operação
logging.info("Obtenção de informações de taxonomia para o DataFrame 'tax_hierar' concluída com sucesso.")


In [ ]:
# Lista temporária para armazenar informações de taxonomia de espécies encontradas
temp_df = []

# Iterando sobre os valores da coluna 'taxonID' do DataFrame 'tax_hierar'
for i, valor in enumerate(tax_hierar['taxonID']):
    # Parâmetros para a solicitação HTTP
    parametros = {'txid': valor, 'rank': 'custom', 'srank': rank_dwc, 'format': 'json'}

    # Realizando uma solicitação GET para a API de taxonomia
    response = requests.get(url_api, params=parametros)
    # Verificando se a resposta foi bem-sucedida
    if response.status_code == 200:
        data = response.json()
        dicta = list(data.values())[0]

        # Iterando sobre os itens retornados pela API
        for chave, valor in dicta.items():
            # Verificando se o valor não contém '_' e se não existe no 'temp_df' ou em 'tax_hierar'
            if '_' not in valor and not any(entry['scientificName'] == valor for entry in temp_df) and valor not in tax_hierar['scientificName'].tolist():
                # Chamando a função 'get_taxonomy_id' para obter informações de taxonomia para o valor atual
                temp_name, temp_txid, temp_rank, temp_pai, erro = get_taxonomy_id(valor)

                # Verificando se ocorreu um erro durante a busca de taxonomia
                if erro:
                    lista_erro.append(erro)
                    logging.warning(f"Erro ao obter informações de taxonomia para '{valor}': {erro}")

                # Adicionando as informações obtidas ao 'temp_df'
                temp_df.append({'scientificName': valor, 'ID_filho': None, 'ID_pai': temp_pai, 'taxonID': temp_txid, 'taxon': temp_rank, 'scientificName_correct': temp_name})
    else:
        logging.error(f"Falha ao acessar a API de taxonomia para o taxonID '{valor}'. Status code: {response.status_code}")

# Concatenando 'tax_hierar' com o 'temp_df' e removendo linhas onde 'taxonID' está vazio
tax_hierar = pd.concat([tax_hierar, pd.DataFrame(temp_df)])
tax_hierar.drop(tax_hierar.loc[tax_hierar['taxonID'] == ''].index, inplace=True)

# Removendo entradas duplicadas com base em 'taxonID' e redefinindo o índice
tax_hierar.drop_duplicates(subset=['taxonID'], inplace=True, ignore_index=True)
tax_hierar.reset_index(drop=True, inplace=True)

logging.info("Concluída a busca e concatenação de informações de taxonomia.")


In [ ]:
# Removendo itens duplicados e vazios da lista de erros
lista_erro = [item for item in list(set(lista_erro)) if item and item not in tax_hierar['scientificName']]

# Lista temporária para armazenar informações de taxonomia de espécies encontradas com erro
temp_df_erro = []

# Loop enquanto houver itens na lista de erros
while len(lista_erro) > 0:
    # Verificando se o primeiro item da lista de erros não está em 'tax_hierar'
    if lista_erro[0] not in tax_hierar['scientificName']:
        # Chamando a função 'get_taxonomy_id' para obter informações de taxonomia para o valor atual
        temp_name, temp_txid, temp_rank, temp_pai, erro = get_taxonomy_id(valor)

        # Verificando se ocorreu um erro durante a busca de taxonomia
        if erro:
            lista_erro.append(erro)

        # Adicionando as informações obtidas ao 'temp_df_erro'
        temp_df_erro.append({'scientificName': valor, 'ID_filho': None, 'ID_pai': temp_pai, 'taxonID': temp_txid, 'taxon': temp_rank, 'scientificName_correct': temp_name})
    lista_erro.pop(0)

# Concatenando 'tax_hierar' com o 'temp_df_erro' e removendo linhas onde 'taxonID' está vazio
tax_hierar = pd.concat([tax_hierar, pd.DataFrame(temp_df_erro)])
tax_hierar.drop(tax_hierar.loc[tax_hierar['taxonID'] == ''].index, inplace=True)

# Removendo entradas duplicadas com base em 'taxonID' e redefinindo o índice
tax_hierar.drop_duplicates(subset=['taxonID'], inplace=True, ignore_index=True)
tax_hierar.reset_index(drop=True, inplace=True)

logging.info("Concluída a busca e concatenação de informações de taxonomia com tratamento de erros.")


In [ ]:
# Dicionário para mapear o ID do pai para o índice correspondente no DataFrame 'tax_hierar'
pai_to_index = {value: index for index, value in enumerate(tax_hierar['taxonID'].unique())}

# Iterando sobre os valores da coluna 'ID_pai' do DataFrame 'tax_hierar'
for i, valor in enumerate(tax_hierar['ID_pai']):
    # Número máximo de tentativas para obter informações de taxonomia
    num_tentativas = 3
    tentativa = 0

    # Verificando se o valor está presente na coluna 'taxonID'
    if valor in tax_hierar['taxonID'].values:
        # Obtendo o índice do pai e atribuindo ao 'parent_taxon_index'
        tax_hierar.loc[i, 'parent_taxon_index'] = tax_hierar.index[tax_hierar['taxonID'] == valor].tolist()[0]
    else:
        # Tentativas para encontrar o pai na hierarquia taxonômica
        while tentativa < num_tentativas:
            # Se o taxon atual for 'kingdom', interrompe a tentativa
            if tax_hierar.loc[i, 'taxon'] == 'kingdom':
                break

            try:
                # Obtendo informações de taxonomia para o ID do pai
                record = efetch_NCBI(str(tax_hierar.loc[i, 'ID_pai']))

                # Obtendo o ID do pai do pai
                taxid_sup_rank = record[0]['LineageEx'][-1]['TaxId']
                tax_hierar.loc[i, 'ID_pai'] = taxid_sup_rank

                # Se o rank do pai for 'kingdom', interrompe a tentativa
                if record[0]['Rank'] == 'kingdom':
                    break

                # Se o ID do pai estiver mapeado, atribui o índice ao 'parent_taxon_index'
                if taxid_sup_rank in pai_to_index:
                    tax_hierar.loc[i, 'parent_taxon_index'] = pai_to_index[taxid_sup_rank]
                    break

            except Exception as e:
                # Se ocorrer um erro, imprime o erro e tenta novamente
                logging.error(f"Erro na tentativa {tentativa + 1}: {str(e)} {valor}")
                tentativa += 1

# Substituindo valores 'None' por 'NaN' na coluna 'parent_taxon_index'
tax_hierar['parent_taxon_index'].replace({None: 'NaN'}, inplace=True)

logging.info("Concluída a atribuição de índices de pais para cada espécie na hierarquia taxonômica.")


In [ ]:
# Agrupando os índices pelo índice do pai
a = tax_hierar.groupby(['parent_taxon_index'], group_keys=True).groups

# Iterando sobre os grupos e seus valores
for chave, valor in a.items():
    # Verificando se a chave está presente nos índices do DataFrame
    if chave in tax_hierar.index:
        # Convertendo os valores para uma string separada por vírgulas e atribuindo ao 'index_filho'
        tax_hierar.loc[chave, 'index_filho'] = ','.join(map(str, valor.tolist()))

        # Obtendo os taxonIDs dos filhos e atribuindo ao 'ID_filho'
        l = list(tax_hierar.loc[valor, 'taxonID'])
        tax_hierar.loc[chave, 'ID_filho'] = ','.join(l)

logging.info("Concluída a atribuição de índices de filhos para cada espécie na hierarquia taxonômica.")


In [ ]:
# Renomeando as colunas 'scientificName' e 'scientificName_correct'
tax_hierar = tax_hierar.rename(columns={'scientificName':'scientificName_search', 'scientificName_correct':'scientificName'})

arquivo_salvar = 'scientificName1.csv'#salvar_arquivo()
if arquivo_salvar:
    # Salvando o DataFrame em um arquivo CSV
    tax_hierar.to_csv(arquivo_salvar, index=False)

    logging.info(f"Salvo o DataFrame em '{arquivo_salvar}' com sucesso.")

else:
    logging.info("Operação de salvamento de arquivo não foi completada.")


In [ ]:
root = tax_hierar[tax_hierar['taxon'] == 'kingdom']

arvore_taxonomica = construir_arvore_taxonomica(tax_hierar, 'scientificName', 'index_filho')
arvore_newick = arvore_para_newick(arvore_taxonomica, 'Eukaryota')
tree = Phylo.read(io.StringIO(arvore_newick), "newick")

# Visualizando a árvore
Phylo.draw(tree)
# Salvar a árvore em formato Newick
Phylo.write(tree, "arvore/arvore_taxonomica.nwk", "newick")
# Salvar a árvore em formato Nexus
Phylo.write(tree, "arvore/arvore_taxonomica.nex", "nexus")
# Salvar a árvore em formato NeXML
Phylo.write(tree, "arvore/arvore_taxonomica.xml", "nexml")
